In [2]:
%pip install pyspark notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 24.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Builddone
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747921 sha256=fa9149fc3575bb4e663bbfb3ade2d0af17caed8cca66c63255f12b1acc75d4ce
  Stored in directory: /Users/dannyxia/Library/Caches/pip/wheels/8f/cb/c0/cc57eb1bf0f9dc87cdaf2b0dbac49e58a210ff68d21d6fc709
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyNotebookApp") \
    .getOrCreate()

sc = spark.sparkContext

25/04/13 23:26:35 WARN Utils: Your hostname, Dannys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.11 instead (on interface en0)
25/04/13 23:26:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 23:26:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
R = sc.parallelize(["hi", "hello", "ok", "no", "yes", "a", "go", "bye"])

In [4]:
R.filter(lambda s: len(s) == 2).count()

4

In [5]:
IN = sc.parallelize(["hello world", "hi there", "hi", "hello there", "goodbye world"])

In [6]:
triplets = IN.flatMap(lambda line: line.split()) \
             .map(lambda word: ((word[0], len(word)), 1)) \
             .reduceByKey(lambda a, b: a + b) \
             .map(lambda kv: (kv[0][0], kv[0][1], kv[1]))

triplets.collect()

[('h', 5, 2), ('h', 2, 2), ('t', 5, 2), ('w', 5, 2), ('g', 7, 1)]

In [13]:
IN = sc.parallelize([(1, 10), (1, 20), (2, 30), (2, 40), (3, 50)])

In [15]:
OUT = IN.mapValues(lambda v: (v, 1)) \
        .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
        .mapValues(lambda x: x[0] / x[1])
OUT.collect()

[(1, 15.0), (2, 35.0), (3, 50.0)]

In [17]:
# Example RDD R
R = sc.parallelize(["The", "The", "is", "a", "test", "Th", "!"])

# Extract the first two letters of each word
result = R.map(lambda word: word[:2]) \
          .map(lambda pair: (pair, 1)) \
          .reduceByKey(lambda a, b: a + b) \
          .filter(lambda pair: pair[0] != '') \
          .sortBy(lambda pair: pair[1], ascending=False)

# Collect the result
result.collect()


[('Th', 3), ('!', 1), ('is', 1), ('a', 1), ('te', 1)]

In [19]:
R = sc.parallelize(["This ice cream is the best!", "Ice cream is delicious!", "The best ice cream!"])

result = R.flatMap(lambda sentence: [(sentence.split()[i], sentence.split()[i + 1]) for i in range(len(sentence.split()) - 1)]) \
          .map(lambda pair: (pair, 1)) \
          .reduceByKey(lambda a, b: a + b) \
          .sortBy(lambda pair: pair[1], ascending=False)

result.collect()

[(('cream', 'is'), 2),
 (('best', 'ice'), 1),
 (('This', 'ice'), 1),
 (('ice', 'cream'), 1),
 (('the', 'best!'), 1),
 (('ice', 'cream!'), 1),
 (('is', 'the'), 1),
 (('is', 'delicious!'), 1),
 (('Ice', 'cream'), 1),
 (('The', 'best'), 1)]